# 플랫폼 업로드를 쉽게하기 위한 로컬 개발 코드
- T3Q.ai(T3Q.cep + T3Q.dl): 빅데이터/인공지능 통합 플랫폼
- 플랫폼 업로드를 쉽게하기 위하여 로컬에서 2개의 파일로 나누어 코드를 개발한다.
- 파일 1_1: 전처리와 학습모델을 개발하는 코드, 전처리 객체나 성능이 검증된 학습모델 객체를 저장한다.
- 파일 1_2: 학습모델을 통해 추론(예측)을 개발하는 코드, 전처리 객체나 학습모델 객체를 불러와서 추론을 수행한다. 

- 파일 1_1(파일명): 1_1_local_platform_iris_classification_preprocess_train_pytorch.ipynb
- 파일 1_2(파일명): 1_2_local_platform_iris_classification_inference_pytorch.ipynb

### 전처리 객체 또는 학습모델 객체
- 전처리 객체나 학습모델 객쳬는 meta_data 폴더 아래에 저장한다.

### 데이터 세트(학습 데이터/테스트 데이터)
- 학습과 테스트에 사용되는 데이터를 나누어 관리한다.
- 학습 데이터: dataset 폴더 아래에 저장하거나 dataset.zip 파일 형태로 저장한다.
- 테스트 데이터: test_dataset 폴더 아래에 저장하거나 test_dataset.zip 파일 형태로 저장한다.

### 로컬 개발 워크플로우(workflow)  
- 로컬 개발 워크플로우를 다음의 4단계로 분리한다.

1. **데이터 세트 준비(Data Setup)**
- 로컬 저장소에서 전처리 및 학습에 필요한 학습 데이터 세트를 준비한다.

2. **데이터 전처리(Data Preprocessing)**
- 데이터 세트의 분석 및 정규화(Normalization)등의 전처리를 수행한다.
- 데이터를 모델 학습에 사용할 수 있도록 가공한다.
- 추론과정에서 필요한 경우, 데이터 전처리에 사용된 객체를 meta_data 폴더 아래에 저장한다.

3. **학습 모델 훈련(Train Model)**
- 데이터를 훈련에 사용할 수 있도록 가공한 뒤에 학습 모델을 구성한다. 
- 학습 모델을 준비된 데이터 세트로 훈련시킨다.
- 정확도(Accuracy)나 손실(Loss)등 학습 모델의 성능을 검증한다.
- 학습 모델의 성능 검증 후, 학습 모델을 배포한다.
- 배포할 학습 모델을 meta_data 폴더 아래에 저장한다.

4. **추론(Inference)**
- 저장된 전처리 객체나 학습 모델 객체를 준비한다.
- 추론에 필요한 테스트 데이터 세트를 준비한다.
- 배포된 학습 모델을 통해 테스트 데이터에 대한 추론을 진행한다. 

In [1]:
# import
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

C:\Users\USER\anaconda3\envs\tf_gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 저장한 모델을 불러오기 위해 Net 클래스를 인스턴스화하고 model 객체를 생성합니다.

class Net(nn.Module):
    # define nn
    def __init__(self):
        super(Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(4, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = self.fc2(X)
        X = self.fc3(X)
        X = self.softmax(X)
        return X
        
model = Net()

In [3]:
# IRIS dataset를 불러온다.
dataset = pd.read_csv('dataset/iris.csv')

# Species을 numerics로 변환한다.
dataset.loc[dataset.iloc[:,-1]=='Iris-setosa', dataset.columns[-1]] = 0
dataset.loc[dataset.iloc[:,-1]=='Iris-versicolor', dataset.columns[-1]] = 1
dataset.loc[dataset.iloc[:,-1]=='Iris-virginica', dataset.columns[-1]] = 2

train_X, test_X, train_y, test_y = train_test_split(dataset[dataset.columns[0:4]].values, dataset.iloc[:,-1].values, test_size=0.2)
train_X = Variable(torch.Tensor(train_X).float())
test_X = Variable(torch.Tensor(test_X).float())

In [4]:
# 학습 모델 준비
PATH = "./iris_model.pth"
model.load_state_dict(torch.load(PATH))
model.eval()

Net(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)

### 6. 추론(Inference)

In [5]:
# test_X를 model에 통과시켜 tensor(Iris-setosa, Iris-versicolor, Iris-virginica)로 이루어진 튜플 predict_out를 출력한다.
# torch.max() 함수를 통해, predict_out의 튜플들 중에서 가장 높은 숫자로 이루어진 1차원 tensor 배열 predict_y를 출력한다.

predict_out = model(test_X)
_, predict_y = torch.max(predict_out, 1)

In [6]:
# tensor(0, 1, 2)를 Iris-setosa, Iris-versicolor, Iris-virginica로 변환해준다.
# 예상값들을 리스트 prediction에 넣어 저장한다.

prediction = []
for data in predict_y:
    if data == 0:
        prediction.append("Iris-setosa")
    elif data == 1:
        prediction.append("Iris-versicolor")
    elif data == 2:
        prediction.append("Iris-virginica")

In [7]:
# 실제값(test_y)과 예상값(predict_y)의 Accuracy를 계산하고 prediction에 저장된 리스트를 출력한다.

print('prediction accuracy: \t', accuracy_score(test_y.tolist(), predict_y))
print(prediction)

prediction accuracy: 	 1.0
['Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-virginica', 'Iris-setosa', 'Iris-versicolor', 'Iris-setosa']
